In [ ]:
from turbo_ml.turbo_ml_experimental import TurboML_Experimental
from turbo_ml.meta_learning import MetaModelGuesser
from datasets import get_iris
import pandas as pd
from captum.attr import IntegratedGradients

pure_data, target = get_iris()
data = pure_data.copy()
data['target'] = target
MODEL_PATH = 'new_model'
tbml = TurboML_Experimental(data, 'target', hpo_enabled=False, guesser=MetaModelGuesser(path=MODEL_PATH))


In [2]:
model = tbml.guesser._meta_model.to('cpu')
model.eval()
input_data = tbml.guesser.last_input.to('cpu')
predictions = model(input_data)[0]
target = max(range(len(predictions)), key=predictions.__getitem__)

In [ ]:
names = pd.read_csv('algorithm_results.csv')
names.drop('name', axis=1, inplace=True)
classifier_name = names.columns[target]
print(f'Chosen classifier name: {classifier_name}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_importances(feature_names, importances):
    plt.figure(figsize=(12,6))
    plt.bar(range(len(importances)), importances, align='center')

    r_name = zip(range(len(importances)), feature_names)
    ticks, names = zip(*filter(lambda x: abs(importances[x[0]]) > 0.001, r_name))

    plt.xticks(ticks, names, rotation='vertical')

for i in range(36):
    target = i
    ig = IntegratedGradients(model)
    attributions = values = ig.attribute(input_data, target=target)
    visualize_importances(list(tbml.dataset_params.keys()), np.mean(attributions.detach().numpy(), axis=0))
    _ = plt.title(names.columns[target])